In [ ]:
import torch
import sys

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'MPS' if torch.backends.mps.is_available() else 'CPU'}")

In [ ]:
import torch
import os
import time
import glob
import json
import shutil
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ultralytics import YOLO
from thop import profile
from pathlib import Path

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"Using device: {device}")

# YOLOv8 Training

Train YOLOv8n model on the dental X-ray dataset.


In [ ]:
# Setup paths
project_root = Path('..').resolve() if Path.cwd().name == 'notebooks' else Path('.').resolve()
processed = project_root / 'data' / 'processed'
models_dir = project_root / 'models'
models_dir.mkdir(parents=True, exist_ok=True)

# Locate dataset
data_yaml_path = processed / 'dataset.yaml'
if not data_yaml_path.exists():
    raise FileNotFoundError("Dataset not found! Run 01_prepare_dataset.ipynb first.")

data_yaml_path = str(data_yaml_path)
test_images_path = Path(data_yaml_path).parent / "test" / "images"

print(f"✓ Data config: {data_yaml_path}")
print(f"✓ Models dir: {models_dir}")

## Train Model


In [ ]:
model_v8 = YOLO('yolov8n.pt')

print("Training YOLOv8n...")
start_time_v8 = time.time()
results_v8 = model_v8.train(
    data=data_yaml_path,
    epochs=100,
    imgsz=640,
    batch=8,
    name='yolov8n_dental',
    device=device,
    verbose=True,  # Shows progress bar with epochs
    plots=False,   # Disable plot generation
    save=True
)
train_time_v8 = time.time() - start_time_v8

path_v8_best_weights = results_v8.save_dir / 'weights' / 'best.pt'
print(f"\n✓ Training complete: {train_time_v8:.2f}s")
print(f"✓ Best weights: {path_v8_best_weights}")

## Profile Model


In [ ]:
print("Profiling model...")
model_v8_trained = YOLO(path_v8_best_weights)

# Move to CPU for THOP compatibility
model_cpu = model_v8_trained.model.cpu()
dummy_input = torch.randn(1, 3, 640, 640)

flops_v8, params_v8 = profile(model_cpu, inputs=(dummy_input,), verbose=False)
print(f"✓ Parameters: {params_v8/1e6:.2f}M | FLOPs: {flops_v8/1e9:.2f}G")

## Measure Inference Time


In [ ]:
print("Measuring inference time...")
# Reload model on correct device for inference
model_v8_trained = YOLO(path_v8_best_weights)

test_images = list(test_images_path.glob('*.jpg')) + list(test_images_path.glob('*.png'))

def get_avg_inference_time(model, image_list):
    if len(image_list) == 0:
        return 0
    _ = model(image_list[0], verbose=False)  # Warm-up
    total_time = 0
    for img in image_list:
        start = time.perf_counter()
        _ = model(img, verbose=False)
        total_time += time.perf_counter() - start
    return (total_time / len(image_list)) * 1000

avg_inf_time_v8 = get_avg_inference_time(model_v8_trained, test_images[:10])
print(f"✓ Avg inference: {avg_inf_time_v8:.2f}ms/image ({len(test_images)} test images)")

In [ ]:
print("Evaluating on test set...")
metrics_v8 = model_v8_trained.val(split='test', data=data_yaml_path, verbose=False)
map50_v8 = metrics_v8.box.map50
map50_95_v8 = metrics_v8.box.map

print(f"✓ mAP@50: {map50_v8:.4f} | mAP@50-95: {map50_95_v8:.4f}")

## Save Results


In [ ]:
print("Saving model and metrics...")
shutil.copy(path_v8_best_weights, models_dir / 'yolov8_best.pt')

metrics_data = {
    'model': 'YOLOv8n',
    'training_time': train_time_v8,
    'params': params_v8 / 1e6,
    'inference_time_ms': avg_inf_time_v8,
    'map50': float(map50_v8),
    'map50_95': float(map50_95_v8)
}

with open(models_dir / 'yolov8_metrics.json', 'w') as f:
    json.dump(metrics_data, f, indent=2)

print(f"\n{'='*60}")
print(f"✓ Model saved: {models_dir}/yolov8_best.pt")
print(f"✓ Metrics saved: {models_dir}/yolov8_metrics.json")
print(f"{'='*60}")

## Visualize Predictions


In [ ]:
print("Visualizing sample predictions...")
sample_images = test_images[:3]
fig, axes = plt.subplots(1, len(sample_images), figsize=(15, 5))
if len(sample_images) == 1:
    axes = [axes]

for idx, img_path in enumerate(sample_images):
    results = model_v8_trained(img_path, verbose=False)
    plotted = results[0].plot()[..., ::-1]  # BGR to RGB
    axes[idx].imshow(plotted)
    axes[idx].axis('off')
    axes[idx].set_title(f"Sample {idx+1}")

plt.tight_layout()
plt.show()
print("\n✅ YOLOv8 training complete!")